In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
from datetime import datetime, timedelta

In [ ]:
def generate_payment_data(n, m, alpha, n0=10, seed=123, total_transactions=45000, date=None, allow_self_loops=False,
                          proportions=None, transaction_shares=None):
    np.random.seed(seed)

    if proportions is None:
        proportions = [0.05, 0.13, 0.17, 0.65]  # Default proportions for K1, K2, K3, K4
    if transaction_shares is None:
        transaction_shares = [0.15, 0.13, 0.22, 0.50]  # Default transaction shares for K1, K2, K3, K4

    # Calculate the number of participants in each group
    num_participants = [int(n * prop) for prop in proportions]
    num_participants[-1] = n - sum(num_participants[:-1])  # Ensure total participants sum to n

    # Initialize the graph with n0 nodes
    G = nx.DiGraph()
    for i in range(n0):
        G.add_node(i, h=1)

    h = np.ones(n0)
    payments = []

    participants = []
    current_id = 0

    # Assign participants to groups
    for group_size in num_participants:
        participants.append(list(range(current_id, current_id + group_size)))
        current_id += group_size

    # Normalize transaction shares
    total_share = sum(transaction_shares)
    normalized_shares = [share / total_share for share in transaction_shares]

    for k in range(n0, n):
        for l in range(m):
            # Preferential attachment for sender
            sender_prob = h / np.sum(h)
            sender = np.random.choice(G.nodes(), p=sender_prob)

            # Update preferential attachment strength for sender
            G.nodes[sender]['h'] += alpha

            if allow_self_loops:
                receiver = np.random.choice(G.nodes(), p=h / np.sum(h))
            else:
                # Avoid self-loops
                receiver = sender
                while receiver == sender:
                    receiver_prob = h / np.sum(h)
                    receiver = np.random.choice(G.nodes(), p=receiver_prob)

            # Update preferential attachment strength for receiver
            G.nodes[receiver]['h'] += alpha

            # Determine the payment value based on group shares
            sender_group = next(i for i, group in enumerate(participants) if sender in group)
            transaction_share = normalized_shares[sender_group]
            payment_value = int(np.random.randint(100000000, 1000000000000) * transaction_share)

            # Add payment (link) to the graph
            if G.has_edge(sender, receiver):
                G[sender][receiver]['weight'] += 1
            else:
                G.add_edge(sender, receiver, weight=1)

            payments.append((sender, receiver, G[sender][receiver]['weight'], payment_value))

        # Add new node with initial preferential attachment strength
        G.add_node(k, h=1)
        h = np.append(h, 1)

    # Generate timestamps for transactions
    if date is None:
        date = datetime.now().date()

    start_time = datetime.combine(date, datetime.strptime('07:00:00', '%H:%M:%S').time())
    end_time = datetime.combine(date, datetime.strptime('19:00:00', '%H:%M:%S').time())
    total_seconds = (end_time - start_time).seconds

    timestamps = []
    for _ in range(total_transactions):
        random_seconds = np.random.randint(0, total_seconds)
        transaction_time = start_time + timedelta(seconds=random_seconds)
        timestamps.append(transaction_time)

    # Ensure the number of payments matches the total_transactions
    num_payments = len(payments)
    if num_payments > total_transactions:
        payments = payments[:total_transactions]
    else:
        additional_payments = total_transactions - num_payments
        for _ in range(additional_payments):
            sender, receiver, weight, payment_value = payments[np.random.randint(0, num_payments)]
            payments.append((sender, receiver, weight + 1, payment_value))

    payment_data = pd.DataFrame(payments, columns=["Sender", "Receiver", "Payments", "Value"])
    payment_data['Timestamp'] = timestamps[:len(payment_data)]

    return G, payment_data

In [ ]:
def calculate_network_parameters(G, payments_df):
    num_nodes = G.number_of_nodes()
    num_links = G.number_of_edges()
    connectivity = num_links / (num_nodes * (num_nodes - 1))
    reciprocity = nx.reciprocity(G)

    degrees = dict(G.degree())
    in_degrees = dict(G.in_degree())
    out_degrees = dict(G.out_degree())

    avg_degree = np.mean(list(degrees.values()))
    max_k_in = max(in_degrees.values())
    max_k_out = max(out_degrees.values())

    num_payments = payments_df["Payments"].sum()
    value_paid = payments_df["Value"].sum()

    params = {
        "Number of nodes": num_nodes,
        "Number of links": num_links,
        "Connectivity": connectivity,
        "Reciprocity": reciprocity,
        "Average Degree (k)": avg_degree,
        "Max (k-in)": max_k_in,
        "Max (k-out)": max_k_out,
        "Number of payments": num_payments,
        "Value paid": value_paid
    }

    return params

In [ ]:
# Parameters
n = 100  # Total number of nodes
m = 70   # Average number of payments per bank
alpha = 0.1  # Strength of preferential attachment
n0 = 10  # Initial number of nodes
seed = 123  # Random seed
average_transactions = 15000  # Average number of transactions in a day
std_dev_transactions = 2500  # Standard deviation for the number of transactions

# Group proportions and transaction shares
proportions = [0.05, 0.13, 0.17, 0.65]  # K1, K2, K3, K4
transaction_shares = [0.15, 0.13, 0.22, 0.50]  # K1, K2, K3, K4

# Date range
start_date = "2045-01-01"
end_date = "2045-01-31"

# Generate payment data for the specified date range
all_payment_data = pd.DataFrame()
current_date = datetime.strptime(start_date, "%Y-%m-%d").date()
end_date = datetime.strptime(end_date, "%Y-%m-%d").date()

while current_date <= end_date:
    total_transactions = int(np.random.normal(average_transactions, std_dev_transactions))
    total_transactions = max(1, total_transactions)  # Ensure at least one transaction
    _, daily_payment_data = generate_payment_data(n, m, alpha, n0, seed, total_transactions, current_date,
                                                  allow_self_loops=False, proportions=proportions,
                                                  transaction_shares=transaction_shares)
    all_payment_data = pd.concat([all_payment_data, daily_payment_data], ignore_index=True)
    current_date += timedelta(days=1)

# Calculate network parameters for the entire month (example calculation for the last day)
payment_network, payment_data = generate_payment_data(n, m, alpha, n0, seed, total_transactions, end_date,
                                                      allow_self_loops=False, proportions=proportions,
                                                      transaction_shares=transaction_shares)
network_params = calculate_network_parameters(payment_network, payment_data)

In [ ]:
# Display the network parameters for the last day of the month
for key, value in network_params.items():
    print(f"{key.ljust(20)}: {value:.3f}")

Number of nodes     : 100.000
Number of links     : 3289.000
Connectivity        : 0.332
Reciprocity         : 0.575
Average Degree (k)  : 65.780
Max (k-in)          : 60.000
Max (k-out)         : 59.000
Number of payments  : 47518.000
Value paid          : 2336893814931504.000


In [ ]:
all_payment_data

,Sender,Receiver,Payments,Value,Timestamp
0,6,2,1,123234817285,2045-01-01 09:21:19
1,5,7,1,69125052246,2045-01-01 10:38:44
2,9,6,1,70442705222,2045-01-01 16:21:08
3,3,7,1,53645027800,2045-01-01 10:16:39
4,0,3,1,44230695328,2045-01-01 14:21:09
...,...,...,...,...,...
478912,8,13,2,82312346112,2045-01-31 11:42:36
478913,35,18,2,399120123390,2045-01-31 14:17:17
478914,1,17,6,89409844693,2045-01-31 14:37:32
478915,0,4,7,149332905506,2045-01-31 14:15:43


In [ ]:
from google.colab import files
all_payment_data.to_csv('data_simulasi_rtgs_jan45_v2.csv',sep = ';')
files.download('data_simulasi_rtgs_jan45_v2.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>